In [29]:
import json
import requests
import time

# Genre based SteamSpy API fallback

In [30]:
def genre_finder_steamspy(game_id):
    try: 
        response = requests.get(
            f"https://steamspy.com/api.php?request=appdetails&appid={game_id}"
        )
        game_data = None
        if response.status_code == 200:
            game_data = response.json()

        if game_data and 'genre' in game_data:
            genre = game_data['genre']
            print(f"Found genre for {game_id}: {genre}")
            return genre
    except Exception as e:
        print(f"Error getting genre for {game_id}: {e}")
        return None

In [31]:
def get_top_genre(genre, exclusion, n_games, max_entry=50):
    """Get top N games from same genre, excluding specified game, usualy self"""
    try:
        response = requests.get("https://steamspy.com/api.php", params={
            "request": "genre", 
            "genre": genre
        })

        # if request fails:
        if response.status_code != 200:
            print(f"API request failed with status {response.status_code}")
            return []
        # if response is empty
        genre_data = response.json() # Games should already be arranged on popularity
        if not genre_data:
            print(f"API Response Empty")
            return []
        
        # Convert to list
        game_items = list(genre_data.items())[:max_entry]  # Limit processing

        # providing recomendations
        recommendations = []
        for app_id_str, game_data in game_items:
            if len(recommendations) >= n_games:
                break
            
            app_id = int(app_id_str)
            if exclusion and app_id == exclusion:
                continue
            
            if game_data.get('name') and game_data.get('developer'):
                recommendations.append({
                    'app_id': app_id,
                    'name': game_data['name'],
                    'developer': game_data.get('developer'),
                    'similarity_score': 0.8  # Default for genre-based
                })
        
        print(f"Found {len(recommendations)} {genre} games from API")
        return recommendations
    except Exception as e:
        print(f"Error getting genre games: {e}")
        return []

# Testing

In [32]:
def test_genre_fallback():
    """Test the genre fallback system with unknown games"""
    
    # Test with some real games first to verify system works
    test_known = [440, 413150, 1030300]  # TF2, Stardew Valley, Hollow Knight: Silksong
    num_rec = 5
    print("\n=== Testing with known games ===")
    for game_id in test_known:
        genre = genre_finder_steamspy(game_id)
        if genre:
            recs = get_top_genre(
                genre, 
                game_id, 
                n_games=num_rec
            )
            print(f"Game {game_id} -> Genre: {genre} -> {len(recs)} recommendations")
            for rec in recs[:num_rec]:
                print(f"  - {rec['name']} ({rec['developer']}): Similarity Score - {rec['similarity_score']}")
        else:
            print(f"No genre found for game {game_id}")
    
    # Test with non-existent games
    test_unknown = [999999, 123456]  # Non-existent games
    print("\n=== Testing with unknown games ===")
    for game_id in test_unknown:
        print(f"Testing fallback for {game_id}")
        genre = genre_finder_steamspy(game_id)
        if genre:
            recs = get_top_genre(genre, exclude_game_id=game_id)
            print(f"Genre: {genre}, Recommendations: {len(recs)}")
        else:
            print(f"No genre found for {game_id}")

# Run the test
if __name__ == "__main__":
    test_genre_fallback()


=== Testing with known games ===
Found genre for 440: Action, Free To Play
Found 5 Action, Free To Play games from API
Game 440 -> Genre: Action, Free To Play -> 5 recommendations
  - Counter-Strike: Global Offensive (Valve): Similarity Score - 0.8
  - Heroes & Generals (TLM Partners): Similarity Score - 0.8
  - Halo Infinite (343 Industries): Similarity Score - 0.8
  - Ring of Elysium (Aurora Studio): Similarity Score - 0.8
  - Black Squad (VALOFE, NS STUDIO): Similarity Score - 0.8
Found genre for 413150: Indie, RPG, Simulation
Found 5 Indie, RPG, Simulation games from API
Game 413150 -> Genre: Indie, RPG, Simulation -> 5 recommendations
  - Project Zomboid (The Indie Stone): Similarity Score - 0.8
  - Don't Starve Together (Klei Entertainment): Similarity Score - 0.8
  - 7 Days to Die (The Fun Pimps): Similarity Score - 0.8
  - Mount & Blade II: Bannerlord (TaleWorlds Entertainment): Similarity Score - 0.8
  - Clicker Heroes (Playsaurus): Similarity Score - 0.8
Found genre for 1030